In [ ]:
import requests
from bs4 import BeautifulSoup
from utils import *
import pandas as pd
import time
from time import sleep
import random
import io 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
import json
import os.path
import networkx as nx
import matplotlib.pyplot as plt
from itertools import combinations

In [ ]:
# generate HTML files and dataframe 
from collector import *

In [ ]:
# generate TSV files from HTML
from parser import *

In [ ]:
# generate all index files and vocabulary
from index import *

In [ ]:
# import main with all we need for the search engine
from main import * 

## Task 1

In [ ]:
search(1, 'world war')

## Task 2 

In [ ]:
search(2, 'Kill Bill', 20)

## Task 3

        We have created additional inverted index for movie names in order to get the first results of search engine with movies containing query terms in their names. In this case, we compute cosine similarity for all documents with at least one query term, thus we have vectors or documents with zero components.

     
       Then we ask the user to insert an additional parameter (year or language) to calculate a new score and have a new ranking based on the distance from the requested year or based on the language entered


In [ ]:
#  Create new index and norm for movie names
name_index, name_norm = create_inverted_index_2([3], vocabulary, norm_file='name_norm.json', index_file='name_index.json')

In [ ]:
doc_norm = load_doc_norm()

In [ ]:
search(3, 'Sherlokc Holmes', 3)

# BONUS PART

In [ ]:
G = nx.Graph()

query = '1980'
search_result = search_with_tfidf(query, vocabulary, inverted_index_2, doc_norm)
for score, doc_id in search_result:
    file_name = f'tsv/{doc_id}.tsv'
    starring = data_from_tsv(file_name, [7]).split()
    #print(starring)
    actors = [t[0] +' '+ t[1] for t in zip(starring[0::2], starring[1::2])]
    G.add_edges_from(combinations(actors, 2))


plt.figure(figsize=(18, 18))
nx.draw(G)
